In [1]:
import geopandas as gpd
import shapely
from shapely.ops import unary_union
from bc_power import utils, hydro
import pandas as pd
import pypsa
import numpy as np
import atlite
import pyomo
import matplotlib.pyplot as plt
import math
pyomo.__version__

'6.5.0'

### Merging shapely polygons

In [ ]:
gdf = gpd.read_file("/mnt/c/Users/pmcw9/Delta-E/PICS/Data/regions/gadm41_CAN_1.json")
mask = gdf['NAME_1'] == "BritishColumbia"
west_lon = gdf[mask].geometry.bounds['minx'].iloc[0]
south_lat = gdf[mask].geometry.bounds['miny'].iloc[0]
east_lon = gdf[mask].geometry.bounds['maxx'].iloc[0]
north_lat = gdf[mask].geometry.bounds['maxy'].iloc[0]

In [ ]:
bbox = (west_lon, south_lat, east_lon, north_lat)
polygon_1 = shapely.geometry.box(*bbox, ccw=True)

bbox_2 = (west_lon-5, south_lat-5, east_lon-5, north_lat-5)
polygon_2 = shapely.geometry.box(*bbox_2, ccw=True)

polys = [polygon_1, polygon_2]

gpd.GeoSeries(polys).boundary.plot()

In [ ]:
west_lon,south_lat,east_lon ,north_lat

### Loading configuration file and testing extraction of year

In [ ]:
config_file = r"/mnt/c/Users/pmcw9/Delta-E/PICS/PyPSA_BC/config/config.yaml"
cfg = utils.load_config(config_file)

In [ ]:
start_year = cfg['cutout']["snapshots"]["start"][0][:4]
end_year = cfg['cutout']["snapshots"]["end"][0][:4]

In [ ]:
prefix = cfg['cutout']['path'] + cfg['cutout']['region']["name"]
suffix = "2021" + ".nc"

"_".join([prefix, suffix])

### Loading BC Hydro Load Data and Community Energy and Emissions Inventory

In [ ]:
bch = pd.read_csv("/mnt/c/Users/pmcw9/Delta-E/PICS/Data/BCH/BalancingAuthorityLoad2020.csv")
ceei_path = "/mnt/c/Users/pmcw9/Delta-E/PICS/Data/CEEI/bc_utilities_energy_and_emissions_data_at_the_community_level.xlsx"
ceei_bch = pd.read_excel(ceei_path,sheet_name="BC Hydro")
ceei_fbc_elec = pd.read_excel(ceei_path,sheet_name="FBC Elec") # Will not use data prior to 2013 otherwise need to pull Kelowna records too
ceei_nw = pd.read_excel(ceei_path,sheet_name="NewWest")
ceei_nel = pd.read_excel(ceei_path,sheet_name="NelsonHydro")
ceei_gf = pd.read_excel(ceei_path,sheet_name="GrandForks")
ceei_pen = pd.read_excel(ceei_path,sheet_name="Penticton")
ceei_sl = pd.read_excel(ceei_path,sheet_name="Summerland")
ceei_prc = pd.read_excel(ceei_path,sheet_name="Princeton")
ceei_yk = pd.read_excel(ceei_path,sheet_name="YukonElec")

In [ ]:
elc_col = "CONSUMPTION_TOTAL" # Units of KW-hr
ceei_list = [ceei_bch, ceei_fbc_elec, ceei_nw, ceei_nel, ceei_gf, ceei_pen, ceei_sl, ceei_prc, ceei_yk] # ceei_fbc_elec
year = 2020
tot = 0 # capture total electricity demand in MW-hr for all ELC demands in the dataset
for df in ceei_list:
    mask = (df['YEAR'] == year) & ((df['ORG_TYPE'] != "Province") | (df["SOURCE"] != "BC Hydro"))
    tot += df[mask][elc_col].sum() / 1000 # convert to MW-hr

ratio = bch['Balancing Authority Load'].sum() / tot
print(f"The ratio between the BC Hydro load data and CEEI load is {ratio}")

In [ ]:
bch['Balancing Authority Load']

### Thermal PP

In [ ]:
tpp_path = "/mnt/c/Users/pmcw9/Delta-E/PICS/Data/SESIT/CODERS/data-pull/supply/generators.csv"
gen_gen_path = "/mnt/c/Users/pmcw9/Delta-E/PICS/Data/SESIT/CODERS/data-pull/supply/generation_generic.csv"
df = pd.read_csv(tpp_path)
gen_params = pd.read_csv(gen_gen_path)

In [ ]:
tpp_gen_types = {'NG_CT', 'NG_CG', 'NG_CC', 'Gas_CT', 'Oil_CT', "Coal", 'Oil_ST', 'Diesel_CT', 'Coal_CCS'}
# BC has only NG_CC, NG_CG, NG_CT
mask = (df["province"] == "BC") & (df["gen_type"].apply(lambda x: x in tpp_gen_types ))
tpp = df[mask].copy()

In [ ]:
gen_params[gen_params['generation_type'] == "NG_CT"].iloc[0]['efficiency']

In [ ]:
gen_params.columns

In [ ]:
tpp['gen_type']

### Testing conceptual cascade in PyPSA
    i) 3 reservoir cascade
    ii) testing spill and discharge

In [ ]:
def get_multi_link_override():
    '''
    Gets the multi-link override. Needed for cascaded hydroelectric.
    '''
    # From PyPSA CHP Example: This ensures we can add 2 outputs for a single link i.e bus0 -> bus_1 AND bus_2
    override_component_attrs = pypsa.descriptors.Dict(
        {k: v.copy() for k, v in pypsa.components.component_attrs.items()}
    )
    override_component_attrs["Link"].loc["bus2"] = [
        "string",
        np.nan,
        np.nan,
        "2nd bus",
        "Input (optional)",
    ]
    override_component_attrs["Link"].loc["efficiency2"] = [
        "static or series",
        "per unit",
        1.0,
        "2nd bus efficiency",
        "Input (optional)",
    ]
    override_component_attrs["Link"].loc["p2"] = [
        "series",
        "MW",
        0.0,
        "2nd bus output",
        "Output",
    ]
    return override_component_attrs

def get_cascade_network(load_data, **kwargs):
    '''
    This function is going to replace function above. This function will feature a better design for adding links
    and other components when finished.
    '''
    # setup the network
    network = pypsa.Network(override_component_attrs=get_multi_link_override())
    network.set_snapshots(range(len(load_data)))
    
    network.add("Bus", "elc", carrier="AC")  # This likely will move
    network.add("Carrier", "water") # this may move since RoR also has water 
    network.add("Carrier", "inflow", co2_emissions=0.0) # likely will move
    network.add("Load", "elc demand", bus="elc", p_set=load_data) # likely will move
    
    for res,components in kwargs.items():
        for name,params in components.items():
            network.add(**params)

    return network

def create_reservoir_dict(df, inflow_data, cascade_name="demo"):
    '''
    Accepts a dataframe of reservoirs and their parameters and
    ordering of the the reservoirs. This can be used to create dictionaries
    which will be used to instantiate the model of the reservoir.
    
    Modifications: Likely need to have inflow in csv files which
                   can be imported later.
    '''
    res_dict = {}
    for idx,row in df.iterrows():
        aid = row['asset_id']
        max_inflow = max(inflow_data[aid])
        res_dict[aid] = {}
        

        # 1) add water_bus
        res_dict[aid]['water bus'] = {"class_name":"Bus",
                                          "name":" ".join([aid,"water bus"]),
                                          "carrier":"water",
                                        }

        # 2) add reservoir_bus
        res_dict[aid]['reservoir bus'] = {"class_name":"Bus",
                                          "name":" ".join([aid,"reservoir bus"]),
                                          "carrier":"water",
                                        }

        # 3) add store_link
        res_dict[aid]['store link'] = {"class_name":"Link",
                                       "name": " ".join([aid,"store link"]),
                                       "bus0": res_dict[aid]['water bus']['name'],
                                       "bus1": res_dict[aid]['reservoir bus']['name'],
                                       "efficiency":1., # mass balance
                                       "p_nom":1000000, # FIX: Should be derived to ensure larger than max(inflow + spill + discharge of any upstream reservoirs)
                                        } 
        
        # 4) get release_link
        res_dict[aid]['release link'] = {"class_name":"Link",
                                       "name": " ".join([aid,"release link"]),
                                       "bus0": res_dict[aid]['reservoir bus']['name'],
                                       "bus1": res_dict[aid]['water bus']['name'],
                                       "efficiency":1., # mass_balance
                                       "p_nom":row["s_capacity"] + row['p_capacity'],
                                        } 
        

        # 5) get reservoir store
        res_dict[aid]['reservoir store'] = {"class_name":"Store",
                                            "name":" ".join([aid,"reservoir store"]),
                                            "bus":res_dict[aid]['reservoir bus']['name'],
                                            "e_nom":row["r_capacity"]
                                            }
        
        # 6) add inflow generator
        res_dict[aid]['inflow generator'] = {"class_name":"Generator",
                                            "name": " ".join([aid,"inflow generator"]),
                                            "bus": res_dict[aid]['reservoir bus']['name'],
                                            "carrier": "inflow",
                                            "efficiency":1., # mass_balance
                                            "p_nom":max_inflow, # max(inflow series)
                                            "p_set":inflow_data[aid],
                                            "p_max_pu":[i / max_inflow if i != 0 else 0 for i in inflow_data[aid]],
                                            "p_min_pu":[i / max_inflow if i != 0 else 0 for i in inflow_data[aid]],
                                            } 

        # Terminal check
        if type(row["downstream"]) == str: # not terminal
            downstream_aid = " ".join([row["downstream"],"water bus"])
        else: # terminal
            downstream_aid = " ".join([cascade_name,"water exit"])
            # Add bus for the terminal reservoir
            res_dict[aid]['terminal bus'] = {"class_name":"Bus",
                                                "name":downstream_aid,
                                                "carrier":"water",
                                                }
            
            # Add spill store
            res_dict[aid]['terminal store'] = {"class_name":"Store",
                                                "name":cascade_name,
                                                "bus":downstream_aid,
                                                "e_nom":10000000 # The max storage needs to retain all possible water in the model horizon
                                                }

        # 7) get discharge link
        res_dict[aid]['discharge link'] = {"class_name":"Link",
                                            "name": " ".join([aid,"discharge link"]),
                                            "bus0": res_dict[aid]['water bus']['name'],
                                            "bus1": row['elc_bus'],
                                            "bus2": downstream_aid,
                                            "marginal_cost":0.0001,
                                            "efficiency":row['p_capacity'] / row['q_rated'], # power conversion 
                                            "efficiency2":1., # mass balance
                                            "p_nom":row['q_rated'], # Should be derived to ensure larger than max(inflow, spill + discharge)
                                            } 
        # 8) get spill link
        res_dict[aid]['spill_link'] = {"class_name":"Link",
                                        "name": " ".join([aid,"spill link"]),
                                        "bus0": res_dict[aid]['water bus']['name'],
                                        "bus1": downstream_aid,
                                        "efficiency":1., # mass_balance
                                        "p_nom":row['s_capacity'], # Should be derived to ensure larger than max(inflow, spill + discharge)
                                        }

    return res_dict

In [ ]:
res_data = [["r1",1,1,10,0,"r2",np.nan,"elc"],
        ["r2",1,1,10,0,"r3","r1","elc"],
        ["r3",1,1,10,0,np.nan,"r2","elc"]]

inflow_data = {"r1":[10,10,10],"r2":[0,0,0],"r3":[0,0,0]} # {"aid":[inflow_series], ...}

load_data = [1,1,1]

# connecting_node_code will need to be added later
# upstream doesn't matter other than to find the head reservoir
# downstream doesn't matter other than to find the terminal reservoir
reservoirs = pd.DataFrame(res_data, columns=["asset_id","q_rated","p_capacity","s_capacity","r_capacity","downstream","upstream","elc_bus"]) 
res_dict = create_reservoir_dict(reservoirs, inflow_data)
network = get_cascade_network(load_data,**res_dict)
reservoirs.head()

In [ ]:
# network.lopf(pyomo=False, solver_logfile="tester.log");
network.optimize()

# create subplots
fig, axes = plt.subplots(nrows=3, ncols=1,figsize=(12,8))
fig.suptitle('Process variables on conceptual cascade system') 
# SOC
ax_soc = network.stores_t.e.plot(ax=axes[0],marker='o')
ax_soc.set_ylabel('State of Charge (SOC) (m^3)')
ax_soc.set_title('SOC vs Time')

# water discharges
col_sel = ["r1 discharge link", "r2 discharge link", "r3 discharge link"]
ax_discharge = network.links_t.p0[col_sel].plot(ax=axes[1],marker='o')
ax_discharge.set_ylabel('Discharge (MW)')

# inflow 
ax_inflow = network.generators_t.p.plot(ax=axes[2],marker='o')
ax_inflow.set_ylabel('Inflow (m^3)')

# Formatting
ax_inflow.set_xticks([0,1,2])
ax_soc.set_xticks([0,1,2])
ax_discharge.set_xticks([0,1,2])

ax_soc.get_shared_x_axes().join(ax_soc, ax_discharge,ax_inflow)
ax_soc.set_xlabel("")
ax_discharge.set_xlabel("")
ax_soc.set_xticklabels([])
ax_discharge.set_xticklabels([])
# ax_soc.sharex(ax_soc, ax_discharge,ax_inflow)

### Testing thermal PP in PyPSA
    i) Dispatch (check)
    ii) Emissions (check) (global)
    iii) Ramping (check)
    iv) UC (check)
    v) Gas-Grid vs no-Grid (check)
    vi) ramp_limit_start_up ()


In [ ]:
def create_tpp_dict(df):
    '''
    Creates a thermal power plant dictionary that is used to add network components of the thermal powerplants.
    '''
    tpp_dict = {}
    for idx,row in df.iterrows():
        aid = row['name']
        tpp_dict[aid] = {}
        # tpp_dict[aid] = row.to_dict()
        # tpp_dict[aid]["class_name"] = "Generator"

        # create link + store representation of generator
        bus_name = " ".join([row['carrier'], "Bus"])
        tpp_dict[aid]= {"class_name":"Link",
                                    "name": " ".join([aid,"gen link"]),
                                    "bus0": bus_name,
                                    "bus1": row['bus'],
                                    "carrier": row['carrier'],
                                    "efficiency":row['efficiency'],
                                    "ramp_limit_up":row["ramp_limit_up"],
                                    "ramp_limit_down":row["ramp_limit_down"],
                                    "p_nom_extendable":False,
                                    "committable":row["committable"],
                                    "min_up_time":row["min_up_time"],
                                    "ramp_limit_start_up":row["ramp_limit_start_up"],
                                    "ramp_limit_shut_down":row["ramp_limit_shut_down"],
                                    "p_nom":row['p_nom'] / row['efficiency'],
                                    "marginal_cost":row['marginal_cost'], # cost per input unit (Need to be careful when combining fuel cost and variable cost 
                                    "p_min_pu":row['p_min_pu'] # watch out for the forced run condition
                                    }
        
    return tpp_dict

def create_tpp_network(load_data,emission_lim, **kwargs):
    '''
    Creates a thermal powerplant network.
    '''
    # setup the network
    network = pypsa.Network() #(override_component_attrs=get_uc_link_override())
    network.set_snapshots(range(len(load_data)))
    
    # Add carriers
    network.add("Carrier", "co2_emissions") # likely will move
    network.add("Carrier", "NG", co2_emissions=1.0) # likely will move (since defined based on infrastructure)

    # Add buses
    network.add("Bus", "elc", carrier="AC")  # This likely will move (since defined based on infrastructure)
    network.add("Bus", "NG Bus", carrier="NG")

    # Add loads
    network.add("Load", "elc demand", bus="elc", p_set=load_data) # likely will move
    
    # Add global constraints
    network.add("GlobalConstraint",
                name="co2 constraint",
                sense="<=",
                carrier_attribute="co2_emissions",
                constant=emission_lim)

    e_fill = 10000
    network.add(class_name="Store",
                name="NG store",
                bus="NG Bus",
                # e_nom_min=-float("inf"),
                # e_nom_max=0,
                e_nom=e_fill,
                e_initial=e_fill,
                # e_nom_extendable=True,
                # e_min_pu=1.0,
                # e_max_pu=0.0
    )
    
    # link store based
    for tpp,params in kwargs.items():
        network.add(**params)

    # generator based
    # for tpp,params in kwargs.items():
    #     network.add(**params)

    return network

In [ ]:
# tpp_data = [['NG_CC', 'elc', 10, 'NG', 10, 0.4, False, 1, 1, 1, 0.2, 0.2],
#             ['NG_CT', 'elc', 10, 'NG', 10, 0.8, False, 1, 1, 1, 0.2, 0.2],
#             ['NG_CG', 'elc', 10, 'NG', 10, 0.7, False, 1, 1, 1, 0.2, 0.2]]
tpp_data = [['NG_CT', 'elc', 20, 'NG', 5, 1, False, 25, 1, 1, 0.5, 1, 1, 1, 0], 
            ['NG_CC', 'elc', 20, 'NG', 10, 1, False, 25, 1, 1, 1, 1, 1, 1, 0]] # Swap 0 -> 10
tpp = pd.DataFrame(tpp_data, columns=['name','bus','p_nom','carrier','marginal_cost','efficiency','committable','start_up_cost','min_up_time','min_down_time','ramp_limit_up','ramp_limit_down','ramp_limit_start_up','ramp_limit_shut_down', "p_min_pu"])
tpp.head()

In [ ]:
load_data = [10, 25, 20] # [10, 15, 9, 15, 15]
emission_lim = 10000 # Model horizon

# ramp limit still has not been added

tpp_dict = create_tpp_dict(tpp)
network = create_tpp_network(load_data,emission_lim, **tpp_dict)
# m = network.optimize.create_model()
# m.solve(solver_name="glpk")

In [ ]:
network.optimize()

In [ ]:
network.links_t.mu_ramp_limit_up

### VRE concept in PyPSA
    i) Generation (check)
    ii) Snapshots (check)
    iii) investments (check)
    

In [ ]:
def create_vre_network(vre_ts,load_ts,vre_dict):
    '''
    Adds a generator to a bus with a time varying load.
    '''
    # setup the network
    network = pypsa.Network()
    network.set_snapshots(load_ts.index)

    # Add buses
    network.add("Bus", "elc", carrier="AC")  # This likely will move (since defined based on infrastructure)
    network.add("Carrier", "NG")
    # Add loads
    network.add("Load", "elc demand", bus="elc", p_set=load_ts) # likely will move
    
    # Add base/peaking generator
    network.add(class_name="Generator",
                name="Filler",
                bus="elc",
                p_nom=10,
                carrier="NG",
                marginal_cost=10)

    for name,params in vre_dict.items():
        network.add(**params)

    return network


def create_vre_dict(vre,vre_ts_dict):
    '''
    Returns a vre_dict
    '''
    vre_dict = {}
    for idx,row in vre.iterrows():
        vre_dict[row["name"]] = {"class_name":row["class_name"],
                                 "name":row["name"],
                                 "bus":row["bus"],
                                 "p_nom":row["p_nom"],
                                 "marginal_cost":row["marginal_cost"],
                                 "p_nom_extendable":row["p_nom_extendable"],
                                 "capital_cost":row["capital_cost"],
                                 "p_max_pu":vre_ts_dict[row["name"]]}
        

    return vre_dict


In [ ]:
# create snapshots
time = pd.date_range('2021-01-01 00:00:00', '2021-01-02 00:00:00', freq='1H', inclusive='left')

# calculation vre
t = np.linspace(0, 2*np.pi, len(time))
y = abs(np.sin(t)) 
load_ts = pd.Series([3]*len(time),index=time)
vre_ts = pd.Series(y, index=time)
vre_ts_dict = {}
vre_ts_dict["sol_generation"] = vre_ts

# create vre df
vre_data = [["Generator", "sol_generation", "elc", 0, 0.0001, True, 50]]
vre = pd.DataFrame(vre_data, columns=["class_name",'name','bus','p_nom',"marginal_cost","p_nom_extendable","capital_cost"])
vre.head()

In [ ]:
vre_dict = create_vre_dict(vre,vre_ts_dict)
network = create_vre_network(vre_ts, load_ts, vre_dict)

In [ ]:
network.optimize()

In [ ]:
network.generators

In [ ]:
vre_ts * 7.5301

In [ ]:
network.generators_t.p

### Loading the network and adding the hydro assets to buses

In [46]:
def is_comp_in_network(comp,network):
    '''
    This function checks to see whether a component has been added already or not.
    '''
    if comp['class_name'] == 'Bus':
        return comp['name'] in network.buses.index
    if comp['class_name'] == 'Link':
        return comp['name'] in network.links.index
    if comp['class_name'] == 'Store':
        return comp['name'] in network.stores.index
    if comp['class_name'] == 'Generator':
        return comp['name'] in network.generators.index
    else:
        return False

In [51]:
network = pypsa.Network(override_component_attrs=utils.get_multi_link_override())
path = r"/home/pmcwhannel/repos/PyPSA_BC/results/pypsa-network"
hydro_ror_path = r"/home/pmcwhannel/repos/PyPSA_BC/results/pypsa-components/hydro_ror.pickle"
hydro_res_path = r"/home/pmcwhannel/repos/PyPSA_BC/results/pypsa-components/hydro_reservoirs.pickle"
network.import_from_csv_folder(path)
ror_dict = utils.read_pickle(hydro_ror_path)
res_dict = utils.read_pickle(hydro_res_path)

INFO:pypsa.io:Imported network pypsa-network has buses, lines, line_types, transformers, transformer_types


In [52]:
network.set_snapshots(ror_dict['BC_ABN_GSS']['p_max_pu'].index)

In [53]:
network.buses

,x,y,type,v_nom,substation_type,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
Bus,,,,,,,,,,,,
138_LMN_GSS,-122.344598,49.568962,138kV,138.0,GSS,AC,None,1.0,0.0,inf,PQ,
138_USR_GSS,-122.350299,49.566047,138kV,138.0,GSS,AC,None,1.0,0.0,inf,PQ,
138_NWS_GSS,-122.462532,49.628056,138kV,138.0,GSS,AC,None,1.0,0.0,inf,PQ,
138_TPA_GSS,-122.162222,49.740918,138kV,138.0,GSS,AC,None,1.0,0.0,inf,PQ,
138_KWL_TSS,-122.205792,49.759520,138kV,138.0,TSS,AC,None,1.0,0.0,inf,PQ,
...,...,...,...,...,...,...,...,...,...,...,...,...
230_WTS_DSS,-117.739732,49.105169,230kV,230.0,DSS,AC,None,1.0,0.0,inf,PQ,
230_BTS_TSS,-117.612786,49.321964,230kV,230.0,TSS,AC,None,1.0,0.0,inf,PQ,
230_ALH_GSS,-117.768849,49.343024,230kV,230.0,GSS,AC,None,1.0,0.0,inf,PQ,


In [54]:
for ror in ror_dict.values():
    network.add(**ror)

for res_comps in res_dict.values():
    for comp in res_comps.values():
        # Need to check if a component has been added already
        if not is_comp_in_network(comp,network):
            network.add(**comp)

/home/pmcwhannel/anaconda3/envs/bc-power/lib/python3.9/site-packages/pypsa/components.py:838: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cls_pnl[k][name] = pd.Series(data=v, index=self.snapshots, dtype=typ)
/home/pmcwhannel/anaconda3/envs/bc-power/lib/python3.9/site-packages/pypsa/components.py:838: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cls_pnl[k][name] = pd.Series(data=v, index=self.snapshots, dtype=typ)
/home/pmcwhannel/anaconda3/envs/bc-power/lib/python3.9/site-packages/pypsa/components.py:838: PerformanceWarning: 

In [10]:
res_dict['BC_BR0_GSS']

{'water bus': {'class_name': 'Bus',
  'name': 'BC_BR0_GSS Water Bus',
  'carrier': 'Water'},
 'reservoir bus': {'class_name': 'Bus',
  'name': 'BC_BR0_RES Water Bus',
  'carrier': 'Water'},
 'store link': {'class_name': 'Link',
  'name': 'BC_BR0_GSS Store Link',
  'bus0': 'BC_BR0_GSS Water Bus',
  'bus1': 'BC_BR0_RES Water Bus',
  'efficiency': 1.0,
  'p_nom': 1012500000.0},
 'release link': {'class_name': 'Link',
  'name': 'BC_BR0_GSS Release Link',
  'bus0': 'BC_BR0_RES Water Bus',
  'bus1': 'BC_BR0_GSS Water Bus',
  'efficiency': 1.0,
  'p_nom': 1302764420.4578984},
 'reservoir store': {'class_name': 'Store',
  'name': 'BC_BR0_RES Reservoir Store',
  'bus': 'BC_BR0_RES Water Bus',
  'e_nom': 1012500000.0},
 'inflow generator': {'class_name': 'Generator',
  'name': 'BC_BR0_RES Inflow Generator',
  'bus': 'BC_BR0_RES Water Bus',
  'carrier': 'inflow',
  'efficiency': 1.0,
  'p_nom': 290264420.4578984,
  'p_set': 2021-01-01 00:00:00    80578.649604
  2021-01-01 01:00:00    80629.005683

In [9]:
res_dict['BC_BR0_DFS']

{'water bus': {'class_name': 'Bus',
  'name': 'BC_BR0_DFS Water Bus',
  'carrier': 'Water'},
 'reservoir bus': {'class_name': 'Bus',
  'name': 'BC_BR0_RES Water Bus',
  'carrier': 'Water'},
 'store link': {'class_name': 'Link',
  'name': 'BC_BR0_DFS Store Link',
  'bus0': 'BC_BR0_DFS Water Bus',
  'bus1': 'BC_BR0_RES Water Bus',
  'efficiency': 1.0,
  'p_nom': 1012500000.0},
 'release link': {'class_name': 'Link',
  'name': 'BC_BR0_DFS Release Link',
  'bus0': 'BC_BR0_RES Water Bus',
  'bus1': 'BC_BR0_DFS Water Bus',
  'efficiency': 1.0,
  'p_nom': 1302764420.4578984},
 'reservoir store': {'class_name': 'Store',
  'name': 'BC_BR0_RES Reservoir Store',
  'bus': 'BC_BR0_RES Water Bus',
  'e_nom': 1012500000.0},
 'inflow generator': {'class_name': 'Generator',
  'name': 'BC_BR0_RES Inflow Generator',
  'bus': 'BC_BR0_RES Water Bus',
  'carrier': 'inflow',
  'efficiency': 1.0,
  'p_nom': 290264420.4578984,
  'p_set': 2021-01-01 00:00:00    80578.649604
  2021-01-01 01:00:00    80629.005683

In [7]:
for res_comps in res_dict.values():
    for comp in res_comps.values():
        if comp["class_name"] == 'Bus':
            print(comp['name'])

BC_ALH_GSS Water Bus
BC_ALH_RES Water Bus
Mica/Columbia Water Bus
BC_ALU_GSS Water Bus
BC_ALU_RES Water Bus
BC_BR0_DFS Water Bus
BC_BR0_RES Water Bus
BC_BR0_GSS Water Bus
BC_BR0_RES Water Bus
BC_GMS_GSS Water Bus
BC_GMS_RES Water Bus
BC_JHT_GSS Water Bus
BC_JHT_RES Water Bus
Campbell Water Bus
BC_LAJ_DFS Water Bus
BC_LAJ_RES Water Bus
BC_LDR_GSS Water Bus
BC_LDR_RES Water Bus
BC_MCA_GSS Water Bus
BC_MCA_RES Water Bus
BC_PCN_GSS Water Bus
BC_PCN_RES Water Bus
BC_REV_GSS Water Bus
BC_REV_RES Water Bus
BC_RUS_GSS Water Bus
BC_RUS_RES Water Bus
Stave Water Bus
BC_SCA_GSS Water Bus
BC_SCA_RES Water Bus
BC_SEV_GSS Water Bus
BC_SEV_RES Water Bus
BC_SFL_GSS Water Bus
BC_SFL_RES Water Bus
BC_SON_DFS Water Bus
BC_SON_RES Water Bus
Bridge Water Bus
BC_STC_GSS Water Bus
BC_STC_RES Water Bus
Peace Water Bus
BC_WAN_GSS Water Bus
BC_WAN_RES Water Bus
Seven Mile Water Bus
BC_WDN_GSS Water Bus
BC_WDN_RES Water Bus
BC_WGS_GSS Water Bus
BC_WGS_RES Water Bus


In [6]:
for res in res_comps

{'water bus': {'class_name': 'Bus',
  'name': 'BC_BR0_GSS Water Bus',
  'carrier': 'Water'},
 'reservoir bus': {'class_name': 'Bus',
  'name': 'BC_BR0_RES Water Bus',
  'carrier': 'Water'},
 'store link': {'class_name': 'Link',
  'name': 'BC_BR0_GSS Store Link',
  'bus0': 'BC_BR0_GSS Water Bus',
  'bus1': 'BC_BR0_RES Water Bus',
  'efficiency': 1.0,
  'p_nom': 1012500000.0},
 'release link': {'class_name': 'Link',
  'name': 'BC_BR0_GSS Release Link',
  'bus0': 'BC_BR0_RES Water Bus',
  'bus1': 'BC_BR0_GSS Water Bus',
  'efficiency': 1.0,
  'p_nom': 1302764420.4578984},
 'reservoir store': {'class_name': 'Store',
  'name': 'BC_BR0_RES Reservoir Store',
  'bus': 'BC_BR0_RES Water Bus',
  'e_nom': 1012500000.0},
 'inflow generator': {'class_name': 'Generator',
  'name': 'BC_BR0_RES Inflow Generator',
  'bus': 'BC_BR0_RES Water Bus',
  'carrier': 'inflow',
  'efficiency': 1.0,
  'p_nom': 290264420.4578984,
  'p_set': 2021-01-01 00:00:00    80578.649604
  2021-01-01 01:00:00    80629.005683

In [5]:
network.buses

,x,y,type,v_nom,substation_type,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
Bus,,,,,,,,,,,,
138_LMN_GSS,-122.344598,49.568962,138kV,138.0,GSS,AC,None,1.0,0.0,inf,PQ,
138_USR_GSS,-122.350299,49.566047,138kV,138.0,GSS,AC,None,1.0,0.0,inf,PQ,
138_NWS_GSS,-122.462532,49.628056,138kV,138.0,GSS,AC,None,1.0,0.0,inf,PQ,
138_TPA_GSS,-122.162222,49.740918,138kV,138.0,GSS,AC,None,1.0,0.0,inf,PQ,
138_KWL_TSS,-122.205792,49.759520,138kV,138.0,TSS,AC,None,1.0,0.0,inf,PQ,
...,...,...,...,...,...,...,...,...,...,...,...,...
BC_ALU_GSS Water Bus,0.000000,0.000000,,1.0,NaN,Water,None,1.0,0.0,inf,PQ,
BC_ALU_RES Water Bus,0.000000,0.000000,,1.0,NaN,Water,None,1.0,0.0,inf,PQ,
BC_BR0_DFS Water Bus,0.000000,0.000000,,1.0,NaN,Water,None,1.0,0.0,inf,PQ,


In [58]:
load_ts = ror_dict['BC_ABN_GSS']['p_max_pu'] * 0.5
network.add("Load", "elc demand", bus="138_LMN_GSS", p_set=load_ts)

In [62]:
network.optimize() # Running all nodes in BC... (Big problem)

INFO:linopy.model: Solve linear problem using Glpk solver
INFO:linopy.io:Writing objective.
Writing variables.: 100%|██████████| 3/3 [00:00<00:00, 91.95it/s]
INFO:linopy.io: Writing time: 0.34s


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --lp /tmp/linopy-problem-tx6isyav.lp --output /tmp/linopy-solve-t_3etg6m.sol
Reading problem data from '/tmp/linopy-problem-tx6isyav.lp'...
94560 rows, 32280 columns, 175224 non-zeros
588164 lines were read
GLPK Simplex Optimizer 5.0
94560 rows, 32280 columns, 175224 non-zeros
Preprocessing...
20784 rows, 21360 columns, 90624 non-zeros
Scaling...
 A: min|aij| =  2.803e-02  max|aij| =  6.156e+02  ratio =  2.196e+04
GM: min|aij| =  2.482e-01  max|aij| =  4.029e+00  ratio =  1.623e+01
EQ: min|aij| =  6.221e-02  max|aij| =  1.000e+00  ratio =  1.608e+01
Constructing initial basis...
Size of triangular part is 16032
      0: obj =  -1.904390867e+09 inf =   3.462e+10 (13716)
   8621: obj =  -1.346355877e+05 inf =   1.228e+07 (6272) 73
  15159: obj =   1.114050848e+05 inf =   1.085e+06 (958) 55
  17536: obj =   1.015966026e+02 inf =   0.000e+00 (0) 13
* 17571: obj =   4.094310439e+01 inf =   0.000e+00 (0)
OPTIMAL LP S

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32280 primals, 94560 duals
Objective: 4.09e+01
Solver model: not available
Solver message: optimal



('ok', 'optimal')

In [66]:
df = network.generators_t.p
threshold = 10
columns_above_threshold = df.columns[df.max() > 0.1]


In [69]:
network.loads_t.p

Load,elc demand
snapshot,
2021-01-01 00:00:00,2.823366
2021-01-01 01:00:00,2.827658
2021-01-01 02:00:00,2.828151
2021-01-01 03:00:00,2.827846
2021-01-01 04:00:00,2.828151
2021-01-01 05:00:00,2.613235
2021-01-01 06:00:00,2.612957
2021-01-01 07:00:00,2.614143
2021-01-01 08:00:00,2.614092


In [68]:
df[columns_above_threshold]

Generator,BC_KMO_GSS RoR Generator
snapshot,
2021-01-01 00:00:00,2.82337
2021-01-01 01:00:00,2.82766
2021-01-01 02:00:00,2.82815
2021-01-01 03:00:00,2.82785
2021-01-01 04:00:00,2.82815
2021-01-01 05:00:00,2.61323
2021-01-01 06:00:00,2.61296
2021-01-01 07:00:00,2.61414
2021-01-01 08:00:00,2.61409


In [74]:
mask1 = network.lines['bus0'].str.contains('KMO')
mask2 = network.lines['bus1'].str.contains('KMO')
network.lines[mask1 | mask2]

,type,bus0,bus1,length,v_nom,s_nom,x,r,g,b,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
MHX_ISS_KMO_GSS,1325,287_MHX_ISS,287_KMO_GSS,75.0,287.0,659.0,24.15,3.0975,0.0,0.00025,...,-inf,inf,0,0.000293,0.000038,0.0,20.611018,0.000293,0.000038,659.0


In [76]:
mask1 = network.lines['bus0'].str.contains('MHX')
mask2 = network.lines['bus1'].str.contains('MHX')
network.lines[mask1 | mask2]

,type,bus0,bus1,length,v_nom,s_nom,x,r,g,b,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
MIN_DSS_MHX_ISS,1325,287_MIN_DSS,287_MHX_ISS,1.751,287.0,659.0,0.563822,0.072316,0.0,0.000006,...,-inf,inf,0,0.000007,8.779553e-07,0.0,0.481199,0.000007,8.779553e-07,659.0
MHX_ISS_KMO_GSS,1325,287_MHX_ISS,287_KMO_GSS,75.000,287.0,659.0,24.150000,3.097500,0.0,0.000250,...,-inf,inf,0,0.000293,3.760517e-05,0.0,20.611018,0.000293,3.760517e-05,659.0


In [77]:
mask1 = network.lines['bus0'].str.contains('MIN')
mask2 = network.lines['bus1'].str.contains('MIN')
network.lines[mask1 | mask2]

,type,bus0,bus1,length,v_nom,s_nom,x,r,g,b,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
MIN_DSS_SKA_DSS,450,230_MIN_DSS,230_SKA_DSS,59.589,230.0,179.0,23.478066,12.990402,0.0,0.000199,...,-inf,inf,0,0.000444,2.455653e-04,0.0,10.517104,0.000444,2.455653e-04,179.0
MIN_DSS_MHX_ISS,1325,287_MIN_DSS,287_MHX_ISS,1.751,287.0,659.0,0.563822,0.072316,0.0,0.000006,...,-inf,inf,0,0.000007,8.779553e-07,0.0,0.481199,0.000007,8.779553e-07,659.0
KIT_DSS_MIN_DSS,632,287_KIT_DSS,287_MIN_DSS,2.503,287.0,314.0,0.938625,0.367190,0.0,0.000008,...,-inf,inf,0,0.000011,4.457868e-06,0.0,0.687858,0.000011,4.457868e-06,314.0
MIN_DSS_EUR_ISS,670,287_MIN_DSS,287_EUR_ISS,1.500,287.0,333.0,0.552000,0.183900,0.0,0.000005,...,-inf,inf,0,0.000007,2.232636e-06,0.0,0.412220,0.000007,2.232636e-06,333.0


In [79]:
mask1 = network.lines['bus0'].str.contains('LMN')
mask2 = network.lines['bus1'].str.contains('LMN')
network.lines.p

AttributeError: 'DataFrame' object has no attribute 'p'

In [83]:
df = network.lines_t.p0
cols = df.columns[df.max() != 0.0]
len(cols)

474